In [1]:
# tf-idf preprocess
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse
train_merged=pd.read_csv('train_merged.csv')
df=train_merged
v = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)
creative_id=df.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=df.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(cr_ad)
sparse.save_npz('cr_ad.npz', cr_ad)

<900000x2533225 sparse matrix of type '<class 'numpy.float64'>'
	with 48745802 stored elements in COOrdinate format>

In [2]:
# cross validation for age
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse

user_info=pd.read_csv('user_info.csv')
X_train=sparse.load_npz('cr_ad.npz')
y_train=user_info["age"]

train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('CountVectorizer_train_age.bin')
display(X_train)

<900000x2533225 sparse matrix of type '<class 'numpy.float64'>'
	with 48745802 stored elements in COOrdinate format>

In [3]:
# cross validation for age
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_age.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':10,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
             }
print("start cv")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    verbose_eval=10, show_stdv=True, seed=0)

start cv
[10]	cv_agg's multi_error: 0.773837 + 0.00108892
[20]	cv_agg's multi_error: 0.772143 + 0.00108821
[30]	cv_agg's multi_error: 0.770558 + 0.00103075
[40]	cv_agg's multi_error: 0.769079 + 0.00107046
[50]	cv_agg's multi_error: 0.768085 + 0.00111039
[60]	cv_agg's multi_error: 0.767147 + 0.00110585
[70]	cv_agg's multi_error: 0.766379 + 0.00116012
[80]	cv_agg's multi_error: 0.765656 + 0.00116009
[90]	cv_agg's multi_error: 0.765029 + 0.00118739
[100]	cv_agg's multi_error: 0.764253 + 0.00120462
[110]	cv_agg's multi_error: 0.763796 + 0.00117341
[120]	cv_agg's multi_error: 0.763329 + 0.00120664
[130]	cv_agg's multi_error: 0.76284 + 0.0011866
[140]	cv_agg's multi_error: 0.762506 + 0.00115682
[150]	cv_agg's multi_error: 0.762025 + 0.00111838
[160]	cv_agg's multi_error: 0.761746 + 0.00107285
[170]	cv_agg's multi_error: 0.761478 + 0.000996448
[180]	cv_agg's multi_error: 0.761132 + 0.00103269
[190]	cv_agg's multi_error: 0.760799 + 0.000942717
[200]	cv_agg's multi_error: 0.760576 + 0.000930784

KeyboardInterrupt: 

In [4]:
# predict age
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_age.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':10,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
              'lambda_l1': 0.2,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=200)
model.save_model("age_model.txt")
print("training completed!")

start training
training completed!


In [1]:
# predict age
%reset -f
# ! pip install lightgbm
import pandas as pd
from scipy import sparse
import lightgbm as lgb
from sklearn import metrics
user_info=pd.read_csv('user_info.csv')
agg=pd.read_csv('agg.csv')
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_test=user_info["age"].iloc[num_train:]
display(y_test)
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='age_model.txt')
y_pred=model.predict(X_test)
display(y_pred)
y_pred.round(0)
print("acc:",metrics.accuracy_score(y_test,y_pred))
print("acc:",metrics.roc_auc_score(y_test,y_pred))

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


720000    2
720001    1
720002    1
720003    2
720004    4
         ..
899995    4
899996    2
899997    3
899998    2
899999    2
Name: age, Length: 180000, dtype: int64

start predicting


array([[0.07672395, 0.21548171, 0.22838336, ..., 0.02785875, 0.01380599,
        0.00671158],
       [0.00976268, 0.07982621, 0.20515466, ..., 0.04191782, 0.01981351,
        0.00462914],
       [0.03804853, 0.17869382, 0.23576319, ..., 0.04120684, 0.02223909,
        0.01036169],
       ...,
       [0.02579289, 0.15219781, 0.23197627, ..., 0.03700645, 0.02087638,
        0.01075989],
       [0.01616925, 0.13558498, 0.23386216, ..., 0.04461791, 0.01773838,
        0.00649407],
       [0.01732835, 0.12255001, 0.24882467, ..., 0.02514533, 0.01428848,
        0.00439648]])

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
y_pred

In [6]:
df=pd.DataFrame(y_pred)

In [8]:
df.describe()

,0
count,180000.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])